In [5]:
from langchain_core.documents import Document
from langchain_community.document_loaders import PDFPlumberLoader

def load(file):
    doc = []
    for f in file:
        loader = PDFPlumberLoader(f)
        doc.extend(loader.load())
    return doc

In [1]:
import glob
import os
path = ''
pdf_files = glob.glob(os.path.join(path, '**', '*.pdf'), recursive=True)
for file in pdf_files:
    print(file)

pdf_file\test1.pdf
pdf_file\test2.pdf
pdf_file\test3.pdf


In [7]:
# PDFPlumnberLoader를 활용하여 불러오기
global_pages = load(pdf_files)

CropBox missing from /Page, defaulting to MediaBox


In [11]:
len(global_pages)

42

In [20]:
global_pages[0].metadata['source']

'test1.pdf'

In [21]:
global_pages[0].metadata['total_pages']

20

In [ ]:
# 조교님 코드 기반으로 실행

from langchain.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader

def process(pdf_files):
    all_pages = []
    for file in pdf_files:
        loader = PyPDFLoader(file)
        pages = loader.load_and_split()

        with open(file,'rb') as f:
            reader = PdfReader(f)
            total_pages = len(reader.pages)
        for page in pages:
            doc = Document(
                page_content = page.page_content,
                metadata = {
                    'source':file,
                    'page':page.metadata.get('page',0),
                    'total_pages':total_pages
                }
            )
            all_pages.append(doc)

    return all_pages

In [24]:
glob_pages = process(pdf_files)

In [26]:
len(glob_pages)

80

In [27]:
glob_pages[0].metadata['total_pages']

20

In [9]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))


In [24]:
# Semantic chuncking 사용
# pip install --q langchain_experimental
# git :https://github.com/FullStackRetrieval-com/RetrievalTutorials
from langchain_experimental.text_splitter import SemanticChunker
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
text_splitter = SemanticChunker(embeddings)

In [25]:
chunks = text_splitter.split_documents(global_pages)

In [26]:
len(chunks)

149

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter_rec = RecursiveCharacterTextSplitter(
        chunk_size = 800,
        chunk_overlap = 100,
    )
chunks_rec = text_splitter.split_documents(global_pages)

In [28]:
len(chunks_rec)

149

In [30]:
# Semantic Chunck 사용시 chunck_size, chunck_overlap 지정 필요가 없다.
chunks == chunks_rec

True